<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
%%capture
!pip install --upgrade tensorflow

In [2]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

2024-04-06 16:00:25.254518: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 16:00:25.421500: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-06 16:00:25.428158: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-06 16:00:25.428200: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

2024-04-06 16:11:52.343146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-06 16:11:52.343212: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-06 16:11:52.343271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-surbhit3812): /proc/driver/nvidia/version does not exist
2024-04-06 16:11:52.343595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 34s 42ms/step - loss: 0.6797 - accuracy: 0.5894 - val_loss: 0.6452 - val_accuracy: 0.6474
Epoch 2/10
782/782 [==============================] - 32s 41ms/step - loss: 0.5986 - accuracy: 0.6884 - val_loss: 0.5701 - val_accuracy: 0.7061
Epoch 3/10
782/782 [==============================] - 32s 41ms/step - loss: 0.5301 - accuracy: 0.7404 - val_loss: 0.5324 - val_accuracy: 0.7285
Epoch 4/10
782/782 [==============================] - 33s 42ms/step - loss: 0.4844 - accuracy: 0.7697 - val_loss: 0.5035 - val_accuracy: 0.7486
Epoch 5/10
782/782 [==============================] - 34s 44ms/step - loss: 0.4519 - accuracy: 0.7911 - val_loss: 0.4873 - val_accuracy: 0.7570
Epoch 6/10
782/782 [==============================] - 36s 46ms/step - loss: 0.4278 - accuracy: 0.8050 - val_loss: 0.4761 - val_accuracy: 0.7647
Epoch 7/10
782/782 [==============================] - 40s 51ms/step - loss: 0.4090 - accuracy: 0.8150 - val_loss: 0.4664 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 7s 9ms/step - loss: 0.4559 - accuracy: 0.7810
Test score: 0.4559166729450226
Test accuracy: 0.7809600234031677


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 80s 100ms/step - loss: 0.6487 - accuracy: 0.6084 - val_loss: 0.5817 - val_accuracy: 0.6907
Epoch 2/10
782/782 [==============================] - 77s 99ms/step - loss: 0.5004 - accuracy: 0.7583 - val_loss: 0.4833 - val_accuracy: 0.7690
Epoch 3/10
782/782 [==============================] - 79s 101ms/step - loss: 0.4235 - accuracy: 0.8114 - val_loss: 0.4440 - val_accuracy: 0.7927
Epoch 4/10
782/782 [==============================] - 78s 99ms/step - loss: 0.3764 - accuracy: 0.8358 - val_loss: 0.4201 - val_accuracy: 0.8065
Epoch 5/10
782/782 [==============================] - 77s 98ms/step - loss: 0.3441 - accuracy: 0.8518 - val_loss: 0.4069 - val_accuracy: 0.8117
Epoch 6/10
782/782 [==============================] - 77s 99ms/step - loss: 0.3185 - accuracy: 0.8654 - val_loss: 0.3948 - val_accuracy: 0.8208
Epoch 7/10
782/782 [==============================] - 77s 98ms/step - loss: 0.3000 - accuracy: 0.8743 - val_loss: 0.3848 - val_accurac

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 73s 92ms/step - loss: 0.6800 - accuracy: 0.5698 - val_loss: 0.6420 - val_accuracy: 0.6530
Epoch 2/10
782/782 [==============================] - 72s 92ms/step - loss: 0.5783 - accuracy: 0.7041 - val_loss: 0.5547 - val_accuracy: 0.7104
Epoch 3/10
782/782 [==============================] - 72s 92ms/step - loss: 0.4912 - accuracy: 0.7655 - val_loss: 0.5081 - val_accuracy: 0.7521
Epoch 4/10
782/782 [==============================] - 72s 92ms/step - loss: 0.4490 - accuracy: 0.7918 - val_loss: 0.4641 - val_accuracy: 0.7799
Epoch 5/10
782/782 [==============================] - 70s 90ms/step - loss: 0.4194 - accuracy: 0.8108 - val_loss: 0.4532 - val_accuracy: 0.7801
Epoch 6/10
782/782 [==============================] - 66s 85ms/step - loss: 0.3964 - accuracy: 0.8228 - val_loss: 0.4291 - val_accuracy: 0.8001
Epoch 7/10
782/782 [==============================] - 64s 82ms/step - loss: 0.3771 - accuracy: 0.8336 - val_loss: 0.4229 - val_accuracy:

In [20]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3257 - accuracy: 0.8612 - val_loss: 0.3792 - val_accuracy: 0.8287
Epoch 2/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3157 - accuracy: 0.8663 - val_loss: 0.3728 - val_accuracy: 0.8328
Epoch 3/10
782/782 [==============================] - 67s 86ms/step - loss: 0.3082 - accuracy: 0.8688 - val_loss: 0.3758 - val_accuracy: 0.8330
Epoch 4/10
782/782 [==============================] - 70s 89ms/step - loss: 0.3019 - accuracy: 0.8719 - val_loss: 0.3626 - val_accuracy: 0.8385
Epoch 5/10
782/782 [==============================] - 69s 88ms/step - loss: 0.2966 - accuracy: 0.8766 - val_loss: 0.3649 - val_accuracy: 0.8367
Epoch 6/10
782/782 [==============================] - 68s 86ms/step - loss: 0.2920 - accuracy: 0.8784 - val_loss: 0.3627 - val_accuracy: 0.8396
Epoch 7/10
782/782 [==============================] - 67s 85ms/step - loss: 0.2889 - accuracy: 0.8803 - val_loss: 0.3640 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation
